In [16]:
import pandas as pd

# Load the data into a DataFrame
df = pd.read_json("sample_supplies_sales.json")

# Extract 'id', 'gender', 'age', and 'saleDate'
df['id'] = df['_id'].apply(lambda x: x['$oid'])
df['gender'] = df['customer'].apply(lambda x: x['gender'])
df['age'] = df['customer'].apply(lambda x: x['age'])
df['saleDate'] = df['saleDate'].apply(lambda x: x['$date'])

# Explode the items
df = df.explode('items').reset_index(drop=True)

# Extract item details into their respective columns
df['price'] = df['items'].apply(lambda x: x['price']['$numberDecimal']).astype(float)
df['quantity'] = df['items'].apply(lambda x: x['quantity'])
df['name'] = df['items'].apply(lambda x: x['name'])

# Drop the original and unnecessary columns
df = df.drop(columns=['_id', 'items', 'customer'])

df


,saleDate,storeLocation,couponUsed,purchaseMethod,id,gender,age,price,quantity,name
0,2015-03-23T21:06:49.506Z,Denver,True,Online,5bd761dcae323e45a93ccfe8,M,42,40.01,2,printer paper
1,2015-03-23T21:06:49.506Z,Denver,True,Online,5bd761dcae323e45a93ccfe8,M,42,35.29,2,notepad
2,2015-03-23T21:06:49.506Z,Denver,True,Online,5bd761dcae323e45a93ccfe8,M,42,56.12,5,pens
3,2015-03-23T21:06:49.506Z,Denver,True,Online,5bd761dcae323e45a93ccfe8,M,42,77.71,2,backpack
4,2015-03-23T21:06:49.506Z,Denver,True,Online,5bd761dcae323e45a93ccfe8,M,42,18.47,2,notepad
...,...,...,...,...,...,...,...,...,...,...
27433,2014-08-18T06:25:49.739Z,New York,False,Online,5bd761deae323e45a93ce36f,M,33,24.12,5,envelopes
27434,2014-08-18T06:25:49.739Z,New York,False,Online,5bd761deae323e45a93ce36f,M,33,27.10,4,pens
27435,2014-08-18T06:25:49.739Z,New York,False,Online,5bd761deae323e45a93ce36f,M,33,29.09,3,binder
27436,2014-08-18T06:25:49.739Z,New York,False,Online,5bd761deae323e45a93ce36f,M,33,598.79,2,laptop


Clean Date Time and arranging the columns

In [19]:
#Convert the 'saleDate' column to a datetime object.
df['saleDate'] = pd.to_datetime(df['saleDate'])

# Extract the date and time from the 'saleDate' column.
df['date'] = df['saleDate'].dt.date
df['time'] = df['saleDate'].dt.time.apply(lambda x: x.strftime('%H:%M:%S'))
# Extract year and month name and form a new column 'year_month_text'
df['month'] = df['saleDate'].dt.strftime('%B')

df = df.drop(columns=['saleDate'])

# Define the column order
cols_order = ['id','month', 'date', 'time', 'name', 'storeLocation', 'purchaseMethod', 'gender', 'age', 'price', 'quantity']

# Reorder the DataFrame columns
df = df[cols_order]

df

,id,month,date,time,name,storeLocation,purchaseMethod,gender,age,price,quantity
0,5bd761dcae323e45a93ccfe8,March,2015-03-23,21:06:49,printer paper,Denver,Online,M,42,40.01,2
1,5bd761dcae323e45a93ccfe8,March,2015-03-23,21:06:49,notepad,Denver,Online,M,42,35.29,2
2,5bd761dcae323e45a93ccfe8,March,2015-03-23,21:06:49,pens,Denver,Online,M,42,56.12,5
3,5bd761dcae323e45a93ccfe8,March,2015-03-23,21:06:49,backpack,Denver,Online,M,42,77.71,2
4,5bd761dcae323e45a93ccfe8,March,2015-03-23,21:06:49,notepad,Denver,Online,M,42,18.47,2
...,...,...,...,...,...,...,...,...,...,...,...
27433,5bd761deae323e45a93ce36f,August,2014-08-18,06:25:49,envelopes,New York,Online,M,33,24.12,5
27434,5bd761deae323e45a93ce36f,August,2014-08-18,06:25:49,pens,New York,Online,M,33,27.10,4
27435,5bd761deae323e45a93ce36f,August,2014-08-18,06:25:49,binder,New York,Online,M,33,29.09,3
27436,5bd761deae323e45a93ce36f,August,2014-08-18,06:25:49,laptop,New York,Online,M,33,598.79,2


Q1- Show top 10 products

In [10]:
df['sales'] = df['quantity'] * df['price']
top_products = df.groupby('name')['sales'].sum().sort_values(ascending=False).head(10)
print(top_products)

name
laptop           6775977.07
backpack          817374.10
pens              581843.27
binder            511644.57
notepad           463615.48
envelopes         376658.49
printer paper     367459.29
Name: sales, dtype: float64


Q2- Show top 3 products by store

In [11]:
# Group by 'storeLocation' and 'name', then sum the sales
grouped = df.groupby(['storeLocation', 'name'])['sales'].sum()

# For each store, get the top 3 products by sales
top_3_products_by_store = grouped.groupby('storeLocation').nlargest(3).reset_index(level=0, drop=True)

print(top_3_products_by_store)


storeLocation  name    
Austin         laptop      1018494.05
               backpack     115217.59
               pens          74622.52
Denver         laptop      1961659.25
               backpack     245679.34
               pens         185008.39
London         laptop      1073098.72
               backpack     144529.14
               pens          92602.30
New York       laptop       694613.32
               backpack      82563.14
               pens          64029.90
San Diego      laptop       462973.58
               backpack      50997.42
               pens          42134.55
Seattle        laptop      1565138.15
               backpack     178387.47
               pens         123445.61
Name: sales, dtype: float64


Q3- Show ranking of each store

In [12]:
#Group by 'storeLocation' and sum sales
store_sales = df.groupby('storeLocation')['sales'].sum()

#Sort stores by sales
sorted_stores = store_sales.sort_values(ascending=False)

#Assign rankings
store_rankings = sorted_stores.rank(ascending=False, method='min').astype(int)

print(store_rankings)


storeLocation
Denver       1
Seattle      2
London       3
Austin       4
New York     5
San Diego    6
Name: sales, dtype: int32


Q4- Show purchases and method by gender 

In [20]:
# Create a pivot table
pivot_table = df.pivot_table(index='gender', columns='purchaseMethod', values='id', aggfunc=pd.Series.nunique, fill_value=0)
# Remove column and index names
pivot_table.columns.name = None
pivot_table.index.name = None
# Print the pivot table
print(pivot_table)


   In store  Online  Phone
F      1430     813    284
M      1389     772    312


Q5- Show monthly total sales

In [15]:
# Group by the month and sum the sales
monthly_sales = df.groupby('month')['sales'].sum()

print(monthly_sales)

month
April        756559.46
August       791577.12
December     871436.60
February     733884.21
January      918041.47
July         847051.30
June         769416.01
March        867736.82
May          878892.77
November     860402.92
October      841007.09
September    758566.50
Name: sales, dtype: float64
